<a href="https://colab.research.google.com/github/Masum06/GenderDebiasing/blob/development2/Gender_Encrypter_Decrypter_cloud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.researchgate.net/publication/337918817_What_You_See_Is_What_You_Learn_Mitigating_Gender_Biases_from_Natural_Language_Processing_Context_through_Gender_Encrypted_Training 

# Gender Encryption

### Installation

In [1]:
import spacy
spacy.__version__

'2.2.4'

In [2]:
!conda info --envs

/bin/bash: conda: command not found


In [3]:
!source activate gender

/bin/bash: activate: No such file or directory


https://github.com/zalandoresearch/flair

https://github.com/huggingface/neuralcoref

In [20]:
!pip install flair==0.4.2
!pip uninstall spacy --yes
!pip install -U spacy==2.1.0
##!python -m spacy download en # SMALL VERSION OF SPACY, FAST DOWNLOAD
##!python -m spacy download en_core_web_md # MEDIUM VERSION OF SPACY
!python -m spacy download en_core_web_lg # LARGE VERSION OF SPACY, SLOW DOWNLOAD
!pip install neuralcoref --no-binary neuralcoref
!pip3 install keras==2.2.4
#!conda install -c conda-forge tensorflow-gpu

     |████████████████████████████████| 143kB 1.4MB/s 
     |████████████████████████████████| 798kB 44.3MB/s 
     |████████████████████████████████| 133kB 55.7MB/s 
     |████████████████████████████████| 1.0MB 47.9MB/s 
  Created wheel for sqlitedict: filename=sqlitedict-1.6.0-cp36-none-any.whl size=14689 sha256=54e783c33ccc56baa2d270fba3a7350036ad9e4f5288ec8b76c267cb5eacc725
  Stored in directory: /root/.cache/pip/wheels/bd/57/d3/907c3ee02d35e66f674ad0106e61f06eeeb98f6ee66a6cc3fe
  Created wheel for segtok: filename=segtok-1.5.7-cp36-none-any.whl size=23258 sha256=6e7a52c1b9d41ec40ec096262bb0515189a2444bd3903975eb309d1ec35964a8
  Stored in directory: /root/.cache/pip/wheels/15/ee/a8/6112173f1386d33eebedb3f73429cfa41a4c3084556bcee254
  Created wheel for mpld3: filename=mpld3-0.3-cp36-none-any.whl size=116679 sha256=2d333fe6b23aa1230a8b29b3248afde5ea8042a72ef27c913fc36a6b72e0418e
  Stored in directory: /root/.cache/pip/wheels/c0/47/fb/8a64f89aecfe0059830479308ad42d62e898a3e3cefdf6ba2

     |████████████████████████████████| 826.9MB 1.3MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.1.0-cp36-none-any.whl size=828255078 sha256=a0a75eade3f488a36ac243c6abe0e6e3c08c88119ccd4ac3d10d3d13aaee06bb
  Stored in directory: /tmp/pip-ephem-wheel-cache-y3qz9oop/wheels/b4/d7/70/426d313a459f82ed5e06cc36a50e2bb2f0ec5cb31d8e0bdf09
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')
     |████████████████████████████████| 317kB 1.3MB/s 
  Found existing installation: Keras 2.2.5
    Uninstalling Keras-2.2.5:
      Successfully uninstalled Keras-2.2.5


In [0]:
!pip uninstall tesnforflow --yes
!pip install tensorflow-gpu==1.14
!pip uninstall keras --yes
!pip install keras==2.2.4

     |████████████████████████████████| 377.0MB 46kB/s 
     |████████████████████████████████| 3.2MB 27.3MB/s 
     |████████████████████████████████| 491kB 49.4MB/s 
ERROR: tensorflow 2.2.0rc1 has requirement tensorboard<2.2.0,>=2.1.0, but you'll have tensorboard 1.14.0 which is incompatible.
ERROR: tensorflow 2.2.0rc1 has requirement tensorflow-estimator<2.3.0,>=2.2.0rc0, but you'll have tensorflow-estimator 1.14.0 which is incompatible.
ERROR: tensorboard 1.14.0 has requirement setuptools>=41.0.0, but you'll have setuptools 39.1.0 which is incompatible.
  Found existing installation: tensorboard 1.11.0
    Uninstalling tensorboard-1.11.0:
      Successfully uninstalled tensorboard-1.11.0
  Found existing installation: tensorflow-estimator 2.2.0rc0
    Uninstalling tensorflow-estimator-2.2.0rc0:
      Successfully uninstalled tensorflow-estimator-2.2.0rc0
  Found existing installation: tensorflow-gpu 1.11.0
    Uninstalling tensorflow-gpu-1.11.0:
      Successfully uninstalled tenso

In [0]:
## Not on Colab
#!conda uninstall tensorflow --yes
#!conda install tensorflow-gpu anaconda --yes
#!conda install -c pytorch pytorch --yes

In [5]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 446259875390410946
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 14976350067121124385
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 7806573026460577106
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 15695549568
locality {
  bus_id: 1
  links {
  }
}
incarnation: 6987467107752020532
physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"
]


In [1]:
from keras import backend
assert len(backend.tensorflow_backend._get_available_gpus()) > 0

Using TensorFlow backend.


ImportError: ignored

In [2]:
import tensorflow as tf
tf.test.gpu_device_name()

ImportError: ignored

In [9]:
import torch
torch.cuda.get_device_name(0)

'Tesla P100-PCIE-16GB'

In [12]:
!nvidia-smi

Sun Mar 29 17:11:27 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P0    37W / 250W |    359MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

### Creating Language Resources

In [0]:
male_pronouns   = ["he",  "him", "his$", "his", "himself"]
female_pronouns = ["she", "her", "her$", "hers", "herself"]
neutral_pronouns= ["zie", "zim", "zir", "zis", "zieself"]
merged_pronouns = ["he/she", "him/her", "his/her", "his/hers", "himself/herself"]

In [0]:
gender_pronouns_dict = {}
gender_honorific_dict = {}

In [0]:
for (g1,g2,g3,g4) in zip(male_pronouns, female_pronouns, neutral_pronouns, merged_pronouns):
    element = {"male": g1.replace("$",""), "female":g2.replace("$",""), "neutral":g3, "merged":g4}
    gender_pronouns_dict[g1] = gender_pronouns_dict[g2] = gender_pronouns_dict[g3] = gender_pronouns_dict[g4] =element

In [0]:
male_hons   =  ["Mr.", "Mr", "Md.", "Md", "Sir", "Lord", "Mister"]
female_hons =  ["Ms.", "Ms", "Mst.", "Mst", "Madam", "Lady", "Miss"]
neutral_hons = ["Mx.", "Mx", "Mx.", "Mx", "Sir/Madam", "Lord/Lady", "Mister/Miss"]
married_hons = ["Mrs.", "Mrs", "Mst.", "Mst", "Madam", "Lady", "Mis'ess"]
merged_hons =  ["Mr./Ms.", "Mr/Ms", "Md./Mst.", "Md/Mst", "Sir/Madam", "Lord/Lady", "Mister/Miss"]

In [0]:
for (h1,h2,h3,h4,h5) in zip(male_hons, female_hons, neutral_hons, married_hons, merged_hons):
    element = {"male": h1, "female":h2, "neutral":h3, "married_fem":h4, "merged":h5}
    gender_honorific_dict[h1] = gender_honorific_dict[h2] = gender_honorific_dict[h3] = \
    gender_honorific_dict[h4] = gender_honorific_dict[h5] = element

### Imports

In [0]:
import re

In [0]:
from flair.data import Sentence
from flair.models import SequenceTagger
#from segtok.segmenter import split_single

In [0]:
import flair
assert flair.__version__=='0.4.2'

In [9]:
from keras.models import load_model

Using TensorFlow backend.


In [10]:
import spacy
import neuralcoref

100%|██████████| 40155833/40155833 [00:04<00:00, 8152800.13B/s] 


### Loading Essentials

In [13]:
!git clone --single-branch --branch development2 https://github.com/Masum06/GenderDebiasing.git

Cloning into 'GenderDebiasing'...
remote: Enumerating objects: 127, done.
remote: Total 127 (delta 0), reused 0 (delta 0), pack-reused 127
Receiving objects: 100% (127/127), 43.20 MiB | 8.82 MiB/s, done.
Resolving deltas: 100% (59/59), done.
Checking out files: 100% (44/44), done.


In [14]:
cd GenderDebiasing

/content/GenderDebiasing


In [0]:
import json

char2idx = 'resources/char2idx.json'
idx2char = 'resources/idx2char.json'
with open(char2idx, 'r') as fp:
    char2idx = json.load(fp)
    
with open(idx2char, 'r') as fp:
    idx2char = json.load(fp)

In [16]:
model_name = 'resources/char_rnn_hsc_model_0.h5'
model = load_model(model_name)

AttributeError: ignored

In [0]:
#nlp = spacy.load('en_core_web_lg') # en_core_web_sm #en_core_web_md
import en_core_web_lg
nlp = en_core_web_lg.load()
neuralcoref.add_to_pipe(nlp)

In [0]:
tagger_ner = SequenceTagger.load('ner')
tagger_pos = SequenceTagger.load('pos')

2019-11-13 23:34:38,859 loading file /home/youngladesh/.flair/models/en-ner-conll03-v0.4.pt
2019-11-13 23:36:36,016 loading file /home/youngladesh/.flair/models/en-pos-ontonotes-v0.2.pt


In [0]:
"hi"

'hi'

### Necessary Methods

Name2Gender

In [0]:
import numpy as np
from keras.preprocessing import sequence

# Converts a name into vector
def name2vectorTest(name):
    name = name.lower()
    new_name = ""
    for char in name:
      if char in char2idx:
        new_name += char
    chars = list(new_name)
    vector = [ char2idx[c] for c in chars ]
    return np.array(vector)

# Converts names to fixed size tensor
def names2tensorTest(names, maxlen=25):
    namelist = [name2vectorTest(name) for name in names]
    return sequence.pad_sequences(np.array(namelist), maxlen=maxlen)  # root of all troubles

def name2gender(name):
  result = model.predict_classes(np.array(names2tensorTest([name.lower()])))[0][0]
  if result:
    return "male"
  else:
    return "female"
  
def isMale(name):
  result = model.predict_classes(np.array(names2tensorTest([name.lower()])))[0][0]
  return result

Storing Names in Dictionary

In [0]:
def store(name, name_found, Name2Key, Key2Name, num_keys): 
    
    if name_found not in Name2Key:
        #global num_keys
        num_keys+=1
        key = "PER_"+str(num_keys)
        #gender = name2gender(name_found)
        alias = None
        element = {"name": name, "key": key, "gender":None, "alias":alias, "is_alias": False, "alias_to": None}
        Name2Key[name_found] = element
        Key2Name[key] = {"name": name, "gender": None, "alias": alias}
    
    if name not in Name2Key:
        element_alias = {"name": name, "is_alias": True, "alias_to": name_found}
        Name2Key[name] = element_alias
        Name2Key[name_found]["alias"] = name
        
    return Name2Key[name_found]["key"], num_keys

# Unit Gender Encryption

In [0]:
def gender_encrypt(s):
  s = ' '.join(s.split())
  doc = nlp(s)
  tokenized_text = ' '.join([token.text for token in doc])
  oracle = []
  coref2name = {}

  # POS TAG
  sent = Sentence(tokenized_text)
  tagger_ner.predict(sent)
  tagger_pos.predict(sent)
  tagged_list = sent.to_tagged_string().split()
  tokens_pos = []
  pos = []
  count = 0
  for i in range(0,len(tagged_list),2):
      tokens_pos.append(tagged_list[i])
      count = count+1
      pos.append(tagged_list[i+1])
      
      if tagged_list[i].lower() in male_pronouns+female_pronouns:
          oracle.append(2)
      elif tagged_list[i+1] in ['<B-PER/NNP>', '<I-PER/NNP>', '<E-PER/NNP>', '<S-PER/NNP>']:
          oracle.append(4)
      else:
          oracle.append(0)

  # COREFERENCE RESOLUTION 
  if len(doc)!=len(tokens_pos):
    tokens_doc = [token.text for token in doc]
    print("doc:", tokens_doc)
    print("pos:", tokens_pos)
    return None, None
  
  coref_stack = []
  name_stack = []
  for i in range(len(doc)):
      token = doc[i]
      if token._.in_coref:
          coref_stack.append(tokens_pos[i])
          if oracle[i] == 4:
              name_stack.append(tokens_pos[i])
          oracle[i] += 1
      else:
          if len(name_stack) > 0:
              name = ' '.join(name_stack)
              coref = ' '.join(coref_stack)
              #name2coref[name] = coref
              coref2name[coref] = name
              name_stack.clear()
          coref_stack.clear()

  # IF THE SENTENCE DOES NOT END WITH A PERIOD OR SPECIAL CHARACTER
  if len(name_stack) > 0:
      name = ' '.join(name_stack)
      name2coref[name] = ' '.join(coref_stack)
      name_stack.clear()
  coref_stack.clear()

  Name2Key = {}
  Key2Name = {}
  encrypted = []
  num_keys = 0
  i = 0
  while i<len(tokens_pos): 
      #print("Oracle ", i, tokens_pos[i])
      if oracle[i] == 2:
          pronoun = tokens_pos[i].lower()
          if pos[i] == '<PRP$>':
              pronoun+="$"
          encrypted.append(gender_pronouns_dict[pronoun]["merged"])
      elif oracle[i] == 3:
          coref = doc[i]._.coref_clusters[0][0].text
          pronoun = tokens_pos[i].lower()
          if pos[i] == '<PRP$>':
              pronoun+="$"
          if coref in coref2name:
            name_found = coref2name[coref]
            key = Name2Key[name_found]["key"]
            if pronoun in male_pronouns:
              Name2Key[name_found]["gender"] = "male"
            else if pronoun in female_pronouns:
              Name2Key[name_found]["gender"] = "female"
            else:
              Name2Key[name_found]["gender"] = "<|unk|>"
            encrypted.append("<|coref|>")
            encrypted.append(gender_pronouns_dict[pronoun]["merged"])
            encrypted.append(key)
          else:
            encrypted.append(gender_pronouns_dict[pronoun]["merged"])
          
      elif oracle[i] in [4,5]:
          if i > 0 and tokens_pos[i-1] in gender_honorific_dict:
            hons = encrypted.pop()
            if hons in male_hons:
              gender = "male"
            else if hons in female_hons+married_hons:
              gender = "female"
            else:
              gender = "<|unk|>"
            encrypted.append("<|hons|>")
            encrypted.append(gender_honorific_dict[hons]["merged"])
          if pos[i] == '<S-PER/NNP>':
              name = tokens_pos[i]
          elif pos[i] == '<B-PER/NNP>':
              name = ""
              while True:
                  #print(i, oracle[i])
                  name += tokens_pos[i]
                  if pos[i] == '<E-PER/NNP>':
                      break
                  name += " "
                  i+=1
          
          if oracle[i] == 4:
              key, num_keys = store(name, name, Name2Key, Key2Name, num_keys)
              encrypted.append(key)
          else:
              coref = doc[i]._.coref_clusters[0][0].text
              name_found = coref2name[coref]
              if name == name_found:
                  key, num_keys = store(name, name_found, Name2Key, Key2Name, num_keys)
                  encrypted.append(key)
              else:
                  key, num_keys = store(name, name_found, Name2Key, Key2Name, num_keys)
                  encrypted.append("<|alias|>")
                  encrypted.append(key)
          Name2Key[key]["gender"] = gender
      else:
        encrypted.append(tokens_pos[i])
      i+=1
      
  encrypted_text = ' '.join(encrypted)

  return tokenized_text, Key2Name, encrypted_text

# Gender Decryption

In [0]:
def gender_decrypt(encrypted_text, Key2Name):
  encrypted = encrypted_text.split()
  decrypted = []
  i = 0
  while i != len(encrypted):
    token = encrypted[i]
    if token in Key2Name:
      decrypted.append(Key2Name[token]["name"])
    elif token == '<|alias|>':
      i+=1
      key = encrypted[i]
      if Key2Name[key]["alias"]:
        decrypted.append(Key2Name[key]["alias"])
      else:
        decrypted.append(Key2Name[key]["name"])
    elif token == '<|coref|>':
      startOfSent = (i==0 or encrypted[i-1] == ".")
      pronoun =  encrypted[i+1]
      key = encrypted[i+2]
      gender = Key2Name[key]["gender"].lower()
      decrypted_pronoun = gender_pronouns_dict[pronoun][gender]
      decrypted_pronoun = decrypted_pronoun[0].upper()+decrypted_pronoun[1:] if startOfSent else decrypted_pronoun
      decrypted.append(decrypted_pronoun)
      i+=2
    elif token == "<|hons|>":
      hons = encrypted[i+1]
      key = encrypted[i+2] # NEED CHECK FOR HONS IN ALIAS
      gender = Key2Name[key]["gender"].lower()
      decrypted.append(gender_honorific_dict[hons][gender])
      i+=1
    elif token == "he/she":
        if i==0 or encrypted[i-1] == ".":
            decrypted.append("He/She")
        else:
            decrypted.append("he/she")
    else:
      decrypted.append(token)
    i+=1

  decrypted_text = ' '.join(decrypted)
  #decrypted_text = decrypted_text.replace(" .", ".")
  return decrypted_text

In [0]:
key2name_changed = {'PER_1': {'name': 'Abira',
    'key': 'PER_1',
    'gender': 'Female',
    'alias': None,
    'is_alias': False,
    'alias_to': None},
  'PER_2': {'name': 'Shibir',
    'key': 'PER_2',
    'gender': 'Male',
    'alias': None,
    'is_alias': False,
    'alias_to': None}
  }

### Testing Encryption-Decryption

In [0]:
s = "Mr. John is not a doctor. Ms. Katy is also not a nurse. John can fly, but she cannot swim."

In [0]:
s = "Morshed Khan told newsmen that 'BNP is a massive party with huge popularity and public acceptance.' This party is now running its operations over Skype. He (Tarique) is operating from London over Skype and that too with only selected leaders. It has turned into the ‘Bangladesh Nationalist Skype Party’. This is painful. I believe the next generation must assume leadership within the party — that would be my recommendation.”"

In [0]:
s = "Dineshwar then left behind his wife’s stabbed body and hanged himself from a tree in a field nearby."

In [0]:
len(s.split())

18

In [0]:
import time 
start = time.time()
text, key2name, encrypted = gender_encrypt(s)
end = time.time()

print(encrypted)
print(key2name)
print(end-start)

PER_1 then left behind <|coref|> his/her PER_1 wife ’s stabbed body and hanged <|coref|> himself/herself PER_1 from a tree in a field nearby .
{'PER_1': {'name': 'Dineshwar', 'gender': 'male', 'alias': None}}
5.390132427215576


In [0]:
key2name = {'PER_1': {'name': 'Dineshwar', 'gender': 'merged', 'alias': None}}

In [0]:
decrypted = gender_decrypt(encrypted, key2name)
decrypted

'Dineshwar then left behind his/her wife ’s stabbed body and hanged himself/herself from a tree in a field nearby .'

In [0]:
decrypted == text

True

In [0]:
en = gender_encrypt("Mr Masum Hasan is not a doctor. Masum is an engineer.")
en

NameError: name 'gender_encrypt' is not defined